<a href="https://colab.research.google.com/github/GirumSe/DataDrive2030-Early-Learning-Predictors-Challenge-my-solution/blob/main/E_starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install eli5, you can use any other model Interpretability libraries
%pip install optuna -q
%pip install eli5 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 KB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 KB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.2/216.2 KB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [19]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
import xgboost as xgb
import lightgbm as lgb
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import cupy as cp
import tensorflow as tf
import eli5
import optuna
import warnings
warnings.filterwarnings('ignore')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [145]:
# Load files
train = pd.read_csv('/content/drive/MyDrive/early_learning/Train.csv')
test = pd.read_csv('/content/drive/MyDrive/early_learning/Test.csv')
ss = pd.read_csv('/content/drive/MyDrive/early_learning/SampleSubmission.csv')

In [ ]:
# Preview train
train.head()

,child_id,data_year,child_date,child_age,child_enrolment_date,child_months_enrolment,child_grant,child_years_in_programme,child_height,child_observe_attentive,...,obs_cooking_5,obs_cooking_6,obs_heating_1,obs_heating_2,obs_heating_3,obs_heating_4,obs_heating_5,obs_heating_6,obs_heating_7,target
0,ID_SYSJ2FM0D,2022.0,2022-02-03,59.000000,NaN,NaN,NaN,NaN,NaN,Sometimes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51.500000
1,ID_J5BTFOZR3,2019.0,NaN,60.163933,NaN,NaN,NaN,1st year in the programme,103.000000,Sometimes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.869999
2,ID_R00SN7AUD,2022.0,2022-03-11,69.000000,NaN,NaN,NaN,NaN,108.400002,Often,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.520000
3,ID_BSSK60PAZ,2021.0,2021-10-13,53.000000,2020-01-15,20.0,No,1st year in the programme,98.099998,Almost always,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58.599998
4,ID_IZTY6TC4D,2021.0,2021-10-13,57.000000,2021-10-13,0.0,NaN,2nd year in programme,114.000000,Almost always,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,76.599998


In [ ]:
# Preview test
test.head()

,child_id,data_year,child_date,child_age,child_enrolment_date,child_months_enrolment,child_grant,child_years_in_programme,child_height,child_observe_attentive,...,obs_cooking_4,obs_cooking_5,obs_cooking_6,obs_heating_1,obs_heating_2,obs_heating_3,obs_heating_4,obs_heating_5,obs_heating_6,obs_heating_7
0,ID_0I0999N6S,2021.0,2021-09-20,57.000000,NaN,NaN,Yes,2nd year in programme,108.0,Almost always,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ID_GQ6ONJ4FP,2021.0,2021-10-21,54.000000,2021-01-10,9.0,Yes,1st year in the programme,105.0,Almost always,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ID_YZ76CVRW3,2021.0,2021-05-17,57.000000,NaN,NaN,Yes,NaN,101.5,Often,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ID_BNINCRXH8,2022.0,2022-09-09,59.334702,NaN,NaN,NaN,3rd year in programme,NaN,Almost always,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,ID_1U7GDTLRI,2021.0,2021-10-12,54.000000,2021-01-15,8.0,Yes,1st year in the programme,103.5,Often,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


handeling 

In [146]:
categorical_cols = ['pri_separate','pri_toys', 'pri_aftercare', 'pri_facilities', 'pri_land', 'pri_facilitiesother', 'pri_landother', 'pri_fundingother', 'pri_bank', 'pri_transport', 'pri_meal_1', 'pri_meal_3', 'pri_meal_4', 'pri_meal_2', 'pri_registered_partial', 'pri_registered_programme', 'pri_registered_npo', 'pri_registered_dsd', 'pri_subsidy', 'pri_network', 'pri_reason_register_year', 'pri_attendance_usual', 'pri_precovid_attendance', 'pri_kitchen', 'pri_funding_6', 'pri_funding_7', 'pri_funding_donations', 'pri_funding_97', 'pri_attendance', 'pri_fees_exceptions', 'pri_education', 'pri_qualification_1', 'pri_qualification_2', 'pri_qualification_3', 'pri_qualification_4', 'pri_qualification_5', 'pri_qualification_6', 'pri_qualification_7', 'pri_qualification_97', 'pri_qualification_0', 'pri_qualificationother', 'pri_founder', 'pri_founderother', 'pri_same_language', 'pri_location', 'pri_locationother', 'pri_library', 'pri_dsd_conditional', 'pri_dsd_conditional_other', 'pri_dsd_unregistered_other', 'pri_registered_health', 'pri_zoning', 'pri_registered_cipc', 'pri_network_type', 'pri_network_type_1', 'pri_network_type_2', 'pri_network_type_3', 'pri_network_type_97', 'pri_name_network_forum', 'pri_email_network_forum', 'pri_name_network_ngo', 'pri_name_network_alliance', 'pri_name_network_other', 'pri_children_4_6_years', 'pri_difficult_see', 'pri_difficult_hear', 'pri_difficult_walk', 'pri_difficult_hold', 'pri_difficult_communicate', 'pri_difficult_learn', 'pri_support_dsd', 'pri_support_dbe', 'pri_support_municipality', 'pri_support_ngo', 'pri_firstaid', 'pri_staff_employed', 'pri_staff_changes_reasons', 'pri_staff_changes_reasonsother', 'pri_covid_staff_retrench', 'pri_covid_staff_salaries', 'pri_money', 'pri_money_1', 'pri_money_2', 'pri_money_3', 'pri_money_97', 'pri_money__1', 'pri_funding_1', 'pri_funding_2', 'pri_funding_3', 'pri_funding_4', 'pri_funding_5', 'pri_funding_none', 'pri_moneyother']
for col in categorical_cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col].astype(str))
    mode_val = train[col].mode()[0] # get the mode value of the column
    train[col].fillna(mode_val, inplace=True) # fill null values with the mode value


In [147]:
categorical_cols = ['pri_separate','pri_toys', 'pri_aftercare', 'pri_facilities', 'pri_land', 'pri_facilitiesother', 'pri_landother', 'pri_fundingother', 'pri_bank', 'pri_transport', 'pri_meal_1', 'pri_meal_3', 'pri_meal_4', 'pri_meal_2', 'pri_registered_partial', 'pri_registered_programme', 'pri_registered_npo', 'pri_registered_dsd', 'pri_subsidy', 'pri_network', 'pri_reason_register_year', 'pri_attendance_usual', 'pri_precovid_attendance', 'pri_kitchen', 'pri_funding_6', 'pri_funding_7', 'pri_funding_donations', 'pri_funding_97', 'pri_attendance', 'pri_fees_exceptions', 'pri_education', 'pri_qualification_1', 'pri_qualification_2', 'pri_qualification_3', 'pri_qualification_4', 'pri_qualification_5', 'pri_qualification_6', 'pri_qualification_7', 'pri_qualification_97', 'pri_qualification_0', 'pri_qualificationother', 'pri_founder', 'pri_founderother', 'pri_same_language', 'pri_location', 'pri_locationother', 'pri_library', 'pri_dsd_conditional', 'pri_dsd_conditional_other', 'pri_dsd_unregistered_other', 'pri_registered_health', 'pri_zoning', 'pri_registered_cipc', 'pri_network_type', 'pri_network_type_1', 'pri_network_type_2', 'pri_network_type_3', 'pri_network_type_97', 'pri_name_network_forum', 'pri_email_network_forum', 'pri_name_network_ngo', 'pri_name_network_alliance', 'pri_name_network_other', 'pri_children_4_6_years', 'pri_difficult_see', 'pri_difficult_hear', 'pri_difficult_walk', 'pri_difficult_hold', 'pri_difficult_communicate', 'pri_difficult_learn', 'pri_support_dsd', 'pri_support_dbe', 'pri_support_municipality', 'pri_support_ngo', 'pri_firstaid', 'pri_staff_employed', 'pri_staff_changes_reasons', 'pri_staff_changes_reasonsother', 'pri_covid_staff_retrench', 'pri_covid_staff_salaries', 'pri_money', 'pri_money_1', 'pri_money_2', 'pri_money_3', 'pri_money_97', 'pri_money__1', 'pri_funding_1', 'pri_funding_2', 'pri_funding_3', 'pri_funding_4', 'pri_funding_5', 'pri_funding_none', 'pri_moneyother']
for col in categorical_cols:
    le = LabelEncoder()
    test[col] = le.fit_transform(test[col].astype(str))

In [148]:
cols_to_fill = ['pri_amount_funding_fees',
                'pri_amount_funding_dsd',
                'pra_measure_rectangle_width',
                'sanitation_educators',
                'pri_fees_amount_2_3',
                'count_staff_paid_practitioners',
                'count_staff_paid',
                'pri_fees_amount_4_6',
                'count_staff_paid_cooks',
                'pra_measure_rectangle_length',
                'count_staff_paid_maintenance',
                'count_staff_paid_assistants',
                'count_staff_paid_support',
                'count_staff_paid_managers',
                'pri_fees_amount_0_1']

# fill null values in each column with its mode value
for col in cols_to_fill:
    mode_val = train[col].mode()[0] # get the mode value of the column
    train[col].fillna(mode_val, inplace=True) # fill null values with the mode value

In [149]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Create an instance of IterativeImputer
imputer = IterativeImputer()

# Fit and transform the imputer on your dataframe with the selected columns
train[['child_observe_total','child_height'
,
'language_match',
'quintile','teacher_social_total']] = imputer.fit_transform(train[['child_observe_total','child_height'
                                                                    ,
                                                                    'language_match',
                                                                    'quintile','teacher_social_total']])


In [150]:
from scipy.stats.mstats import winsorize

# Set the winsorization limits
limits = [0.05, 0.05]

# Find the numeric columns in the dataframe
numeric_cols = train.select_dtypes(include='number').columns

# Iterate over each numeric column
for column in train.columns:
    if column != 'target' and column in numeric_cols:
        column_values = train[column].values
        winsorized_values = winsorize(column_values, limits)
        if not (column_values == winsorized_values).all():
            print(f"Number of extreme values in {column}: {len(column_values) - len(winsorized_values)}")
            train[column] = winsorized_values

Number of extreme values in child_age: 0
Number of extreme values in child_months_enrolment: 0
Number of extreme values in child_height: 0
Number of extreme values in child_observe_total: 0
Number of extreme values in child_zha: 0
Number of extreme values in child_attendance: 0
Number of extreme values in ward_best: 0
Number of extreme values in id_enumerator: 0
Number of extreme values in id_facility: 0
Number of extreme values in pqa_class_assistants: 0
Number of extreme values in pqa_score_environment: 0
Number of extreme values in pqa_score_assessment: 0
Number of extreme values in pqa_score_relationships: 0
Number of extreme values in pqa_score_curriculum: 0
Number of extreme values in pqa_score_teaching: 0
Number of extreme values in pra_experience: 0
Number of extreme values in pra_class_present: 0
Number of extreme values in pra_class_size: 0
Number of extreme values in pra_class_attendance: 0
Number of extreme values in pra_class_attendance_precovid: 0
Number of extreme values

In [151]:
# For a simple model model we will use only numerical columns
y = train.target
num_cols = list(set(test.select_dtypes('number')).intersection(train.select_dtypes('number')))
num = num_cols
X = train[num_cols]
X = X.fillna(0)
ch = test.child_id
test = test[num_cols]
test = test.fillna(0)

In [152]:
scaler = StandardScaler()
X = scaler.fit_transform(X)
test = scaler.transform(test)

Spliting train and test data

In [153]:
# Split data into training and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

MODELS TO TRAIN

In [123]:
from sklearn.model_selection import KFold

# Define the XGBoost parameters
params = {
    'objective': 'reg:squarederror',
    'tree_method': 'gpu_hist',
    'gpu_id': 0,
    'learning_rate': 0.3358058021461148,
    'max_depth': 8,
    'min_child_weight': 10,
    'subsample': 0.7820237238067945,
    'random_state': 42
}

# Define the number of folds
num_folds = 7

# Initialize the K-fold cross-validator
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Initialize arrays to store the results
train_scores = np.zeros(num_folds)
test_scores = np.zeros(num_folds)
predsx = np.zeros(len(test))

# Loop over each fold
for i, (train_index, test_index) in enumerate(kf.split(X)):
    print(f'Fold {i+1}')

    # Split the data into train and test sets
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Create DMatrix objects
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    # Train the model
    xgb_reg = xgb.train(params, dtrain)

    # Make predictions on the train and test sets
    y_train_pred = xgb_reg.predict(dtrain)
    y_test_pred = xgb_reg.predict(dtest)

    # Calculate and print the train and test scores
    train_scores[i] = mean_squared_error(y_train, y_train_pred, squared=False)
    test_scores[i] = mean_squared_error(y_test, y_test_pred, squared=False)
    print(f'Train score: {train_scores[i]}')
    print(f'Test score: {test_scores[i]}')

    # Make predictions on the test set
    dtest = xgb.DMatrix(test)
    predsx += xgb_reg.predict(dtest)

# Calculate the mean prediction
predsx /= num_folds

# Print the final score
print(f'Mean test score: {test_scores.mean()}')

Fold 1
Train score: 8.688554456493153
Test score: 11.225376502071468
Fold 2
Train score: 8.664732060447095
Test score: 10.85587752731087
Fold 3
Train score: 8.724974281840225
Test score: 11.185056891146338
Fold 4
Train score: 8.840017006852896
Test score: 11.048191084677606
Fold 5
Train score: 8.557039615364785
Test score: 11.186674579127386
Fold 6
Train score: 8.667609882469213
Test score: 10.78910082206893
Fold 7
Train score: 8.649549061801624
Test score: 11.010827814988247
Mean test score: 11.043015031627263


In [180]:
from sklearn.model_selection import KFold

# Define parameters for LightGBM model
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'num_leaves': 96, 'learning_rate': 0.08862700152691017, 'subsample': 0.896323925270115, 'subsample_freq': 3, 'colsample_bytree': 0.5194569601525184, 'max_bin': 844, 'max_depth': 20, 'min_child_samples': 27,
    'verbose': -1,
    'random_state': 42
}

# Define K-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Initialize an array to store predictions for each fold
test_predls = np.zeros((len(test),))
test_preds = np.zeros((len(X_test),))
# Loop over the folds
for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    
    # Split data into training and validation sets
    X_train, y_train = X[train_idx], y[train_idx]
    X_val, y_val = X[val_idx], y[val_idx]

    # Fit LightGBM model on training set
    lig_reg = lgb.LGBMRegressor(**params)
    lig_reg.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=50, verbose=0)

    # Make predictions on validation set and calculate RMSE
    val_pred = lig_reg.predict(X_val)
    val_rmse = mean_squared_error(y_val, val_pred, squared=False)
    print(f'Fold {fold}: RMSE on validation set: {val_rmse:.4f}')

    # Make predictions on test set for this fold
    fold_test_preds = lig_reg.predict(X_test)
    test_preds += fold_test_preds
    fold_test_predls = lig_reg.predict(test)
    test_predls += fold_test_predls
    
# Take average of predictions across all folds
test_predls /= kf.n_splits
test_preds /= kf.n_splits
# Make final predictions on test set
predsl = test_predls

# Calculate RMSE on validation set
val_rmse_final = mean_squared_error(y_test, test_preds, squared=False)
print(f'Final RMSE on validation set: {val_rmse_final:.4f}')

Fold 0: RMSE on validation set: 10.1643
Fold 1: RMSE on validation set: 10.0698
Fold 2: RMSE on validation set: 10.1602
Fold 3: RMSE on validation set: 10.4042
Fold 4: RMSE on validation set: 9.9254
Fold 5: RMSE on validation set: 10.0164
Fold 6: RMSE on validation set: 10.3144
Fold 7: RMSE on validation set: 9.7148
Fold 8: RMSE on validation set: 10.0531
Fold 9: RMSE on validation set: 10.2702
Final RMSE on validation set: 6.3913


In [130]:
from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

# Define K-fold cross-validation
kf = KFold(n_splits=7, shuffle=True, random_state=42)

# Initialize an array to store predictions for each fold
test_preds = np.zeros((len(X_test),))
test_predgs = np.zeros((len(test), kf.n_splits))

# Loop over the folds
for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    
    # Split data into training and validation sets
    X_train, y_train = X[train_idx], y[train_idx]
    X_val, y_val = X[val_idx], y[val_idx]
    
    # Fit GradientBoostingRegressor model on training set
    gb_reg = GradientBoostingRegressor(n_estimators=150, max_depth=6)
    gb_reg.fit(X_train, y_train)

    # Make predictions on validation set and calculate RMSE
    val_pred = gb_reg.predict(X_val)
    val_rmse = mean_squared_error(y_val, val_pred, squared=False)
    print(f'Fold {fold}: RMSE on validation set: {val_rmse:.4f}')

    # Make predictions on test set for this fold
    fold_test_preds = gb_reg.predict(X_test)
    test_preds += fold_test_preds
    fold_test_predgs = gb_reg.predict(test)
    test_predgs[:,fold] = fold_test_predgs
    
# Take average of predictions across all folds
test_preds /= kf.n_splits
test_predgs_mean = np.mean(test_predgs, axis=1)

# Calculate final RMSE on test set
test_rmse = mean_squared_error(y_test, test_preds, squared=False)
print(f'Final RMSE on test set: {test_rmse:.4f}')

Fold 0: RMSE on validation set: 10.3629
Fold 1: RMSE on validation set: 10.0796
Fold 2: RMSE on validation set: 10.2057
Fold 3: RMSE on validation set: 10.1810
Fold 4: RMSE on validation set: 10.5520
Fold 5: RMSE on validation set: 9.9347
Fold 6: RMSE on validation set: 10.5602
Final RMSE on test set: 7.2016


In [16]:
def objective(trial):
    # Define hyperparameters to optimize
    params = {
        'objective': 'reg:squarederror',
        'tree_method': 'gpu_hist',
        'gpu_id': 0,
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 1.0),
        'max_depth': trial.suggest_int('max_depth', 1, 500),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_uniform('subsample', 0.01, 1.0),
        'random_state': 42
    }
    # Train model and make predictions
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dvalid = xgb.DMatrix(X_test, label=y_test)
    model = xgb.train(params, dtrain, evals=[(dvalid, 'validation')], early_stopping_rounds=20, verbose_eval=False)
    y_pred = model.predict(dvalid)
    
    # Compute and return RMSE score
    y_test_np = cp.asnumpy(y_test)
    y_pred_np = cp.asnumpy(y_pred)
    return mean_squared_error(y_test_np, y_pred_np, squared=False)

# Define search space and start optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=1000)

# Print best parameters and score
print(f'Best RMSE score: {study.best_value:.3f}')
print(f'Best parameters: {study.best_params}')

[I 2023-04-10 09:06:25,009] A new study created in memory with name: no-name-47b732e6-f008-465a-a117-1d631746d000
[I 2023-04-10 09:06:25,217] Trial 0 finished with value: 50.031863714798604 and parameters: {'learning_rate': 0.0013115342562815605, 'max_depth': 126, 'min_child_weight': 9, 'subsample': 0.8289041288965212}. Best is trial 0 with value: 50.031863714798604.
[I 2023-04-10 09:06:25,513] Trial 1 finished with value: 14.775910293498578 and parameters: {'learning_rate': 0.1596058376411394, 'max_depth': 415, 'min_child_weight': 5, 'subsample': 0.5268452228599528}. Best is trial 1 with value: 14.775910293498578.
[I 2023-04-10 09:06:25,629] Trial 2 finished with value: 47.0523932702408 and parameters: {'learning_rate': 0.007944051357323166, 'max_depth': 99, 'min_child_weight': 7, 'subsample': 0.5498039698034777}. Best is trial 1 with value: 14.775910293498578.
[I 2023-04-10 09:06:25,704] Trial 3 finished with value: 46.679663489428506 and parameters: {'learning_rate': 0.0090501893704

Best RMSE score: 10.862
Best parameters: {'learning_rate': 0.32584447303454683, 'max_depth': 337, 'min_child_weight': 9, 'subsample': 0.9564074163351378}


In [13]:
params = {'objective': 'reg:squarederror', 'tree_method': 'gpu_hist','gpu_id': 0,'learning_rate': 0.3358058021461148, 'max_depth': 8, 'min_child_weight': 10, 'subsample': 0.7820237238067945, 'random_state': 42}
dtrain = xgb.DMatrix(X_train, label=y_train)
xgb_reg = xgb.train(params, dtrain)

# Make predictions using the trained model
dtest = xgb.DMatrix(X_test)
y_pred = xgb_reg.predict(dtest)
y_test_np = cp.asnumpy(y_test)
y_predx = cp.asnumpy(y_pred)
mean_squared_error(y_test_np, y_predx, squared=False)

10.93819060206564

In [20]:
def objective(trial):
    params = {
        'metric': 'mse',
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.001, 0.1),
        'subsample': trial.suggest_uniform('subsample', 0.4, 1.0),
        'subsample_freq': trial.suggest_int('subsample_freq', 1, 10),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.4, 1.0),
        'max_bin': trial.suggest_int('max_bin', 10, 1000),
        'max_depth': trial.suggest_int('max_depth', 1, 20),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 100),
        'random_state': 42,
        'n_jobs': -1,
        'verbose': -1,
    }
    
    model = lgb.LGBMRegressor(**params)
    model.fit(X_train, y_train)
    
    preds = model.predict(X_test)
    score = mean_squared_error(y_test, preds, squared=False)
    
    return score

# Create study and optimize
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=200)

# Print best hyperparameters and score
best_params = study.best_params
best_score = study.best_value
print(f'Best score: {best_score}\nBest params: {best_params}')

[I 2023-04-10 09:20:26,190] A new study created in memory with name: no-name-7ab25aee-8adf-49fd-ab59-b62f67d67a88
[I 2023-04-10 09:20:28,058] Trial 0 finished with value: 10.339516727413683 and parameters: {'num_leaves': 193, 'learning_rate': 0.0639409566544329, 'subsample': 0.6298075421663791, 'subsample_freq': 2, 'colsample_bytree': 0.47236143938329345, 'max_bin': 159, 'max_depth': 17, 'min_child_samples': 13}. Best is trial 0 with value: 10.339516727413683.
[I 2023-04-10 09:20:28,924] Trial 1 finished with value: 10.67839660878803 and parameters: {'num_leaves': 141, 'learning_rate': 0.08383361286774459, 'subsample': 0.4445683647360434, 'subsample_freq': 8, 'colsample_bytree': 0.4308710969218931, 'max_bin': 54, 'max_depth': 11, 'min_child_samples': 7}. Best is trial 0 with value: 10.339516727413683.
[I 2023-04-10 09:20:29,757] Trial 2 finished with value: 10.528582486206046 and parameters: {'num_leaves': 36, 'learning_rate': 0.05785923790390728, 'subsample': 0.5918800408704856, 'subs

Best score: 10.090768116503096
Best params: {'num_leaves': 64, 'learning_rate': 0.09461319875237248, 'subsample': 0.9547225689874418, 'subsample_freq': 6, 'colsample_bytree': 0.45957229392311705, 'max_bin': 173, 'max_depth': 11, 'min_child_samples': 29}


In [25]:
import optuna
import xgboost as xgb
import cupy as cp
from sklearn.metrics import mean_squared_error

def objective(trial):
    # Define hyperparameters to optimize
    params = {
        'metric': 'mse',
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.001, 0.1),
        'subsample': trial.suggest_uniform('subsample', 0.4, 1.0),
        'subsample_freq': trial.suggest_int('subsample_freq', 1, 10),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.4, 1.0),
        'max_bin': trial.suggest_int('max_bin', 10, 1000),
        'max_depth': trial.suggest_int('max_depth', 1, 20),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 100),
        'random_state': 42,
        'n_jobs': -1,
        'verbose': -1,
    }
    
    model = lgb.LGBMRegressor(**params)
    model.fit(X_train, y_train)
    
    preds = model.predict(X_test)
    score = mean_squared_error(y_test, preds, squared=False)
    return score

# Define search space and start optimization using TPE sampler
sampler = optuna.samplers.TPESampler(seed=42)
study = optuna.create_study(direction='minimize', sampler=sampler)
study.optimize(objective, n_trials=200)

# Print best parameters and score
print(f'Best RMSE score: {study.best_value:.3f}')
print(f'Best parameters: {study.best_params}')

[I 2023-04-10 09:33:36,246] A new study created in memory with name: no-name-cefbf4bc-7363-435e-8d60-e4ef0cc2949c
[I 2023-04-10 09:33:36,608] Trial 0 finished with value: 12.416695691596924 and parameters: {'num_leaves': 3, 'learning_rate': 0.025557044224186764, 'subsample': 0.8707533737483772, 'subsample_freq': 1, 'colsample_bytree': 0.9079288656828632, 'max_bin': 266, 'max_depth': 17, 'min_child_samples': 34}. Best is trial 0 with value: 12.416695691596924.
[I 2023-04-10 09:33:37,421] Trial 1 finished with value: 10.576701960830922 and parameters: {'num_leaves': 124, 'learning_rate': 0.07074393363352821, 'subsample': 0.6990724462541142, 'subsample_freq': 3, 'colsample_bytree': 0.76894358623342, 'max_bin': 464, 'max_depth': 10, 'min_child_samples': 72}. Best is trial 1 with value: 10.576701960830922.
[I 2023-04-10 09:33:38,330] Trial 2 finished with value: 10.431700425887906 and parameters: {'num_leaves': 197, 'learning_rate': 0.07382355927591232, 'subsample': 0.8066273888796271, 'sub

Best RMSE score: 10.138
Best parameters: {'num_leaves': 125, 'learning_rate': 0.09268367041562957, 'subsample': 0.8973581541386146, 'subsample_freq': 1, 'colsample_bytree': 0.528584413467353, 'max_bin': 863, 'max_depth': 17, 'min_child_samples': 12}


In [159]:
params = {
    'metric': 'mse',
    'boosting_type': 'gbdt',
    'num_leaves': 96, 'learning_rate': 0.08862700152691017, 'subsample': 0.896323925270115, 'subsample_freq': 3, 'colsample_bytree': 0.5194569601525184, 'max_bin': 844, 'max_depth': 20, 'min_child_samples': 27
}
lig_reg = lgb.LGBMRegressor(**params, random_state=42)
lig_reg.fit(X_train, y_train)
lig_reg_pred = lig_reg.predict(X_test)
mean_squared_error(y_test, lig_reg_pred, squared=False)

10.086815750157122

In [160]:
# Gradient Boosting Regression
gb_reg = GradientBoostingRegressor(n_estimators=290, max_depth=9)
gb_reg.fit(X_train, y_train)
gb_reg_pred = gb_reg.predict(X_test)
mean_squared_error(y_test, gb_reg_pred, squared=False)

10.580034631989363

In [181]:
# Preview overall top 10 features
eli5.show_weights(lig_reg, feature_names=num)

Weight,Feature
0.2058,child_observe_total
0.1167,child_age
0.0709,id_enumerator
0.0457,id_ward
0.0398,teacher_emotional_total
0.0373,child_height
0.0349,latitude
0.0337,longitude
0.0302,id_facility
0.0254,child_zha


In [182]:
eli5.show_weights(gb_reg, feature_names=num)

Weight,Feature
0.2009 ± 0.1070,child_observe_total
0.1651 ± 0.1157,child_age
0.0507 ± 0.0869,child_height
0.0479 ± 0.0429,teacher_emotional_total
0.0459 ± 0.0847,id_enumerator
0.0394 ± 0.0718,id_facility
0.0386 ± 0.1405,child_zha
0.0312 ± 0.0325,id_ward
0.0244 ± 0.0659,teacher_social_total
0.0232 ± 0.0788,latitude


In [ ]:
# Select a single/specific prediction
sample = X_train.iloc[[1]]
sample

,pqa_score_teaching,obs_heating_2,pra_class_size,pri_difficult_hear,obs_heating_3,count_practitioners_age_2,obs_cooking_3,pra_hhsize,language_match,pri_dsd_year,...,latitude,pra_class_attendance_precovid,count_staff_contract,count_toilets_children,count_register_gender_male,count_staff_time_part,pri_expense_other,pqa_class_assistants,count_practitioners_all,count_staff_gender_male
1669,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,-33.833136,0.0,5.666667,0.666667,38.666667,0.0,0.0,0.0,0.0,0.333333


In [183]:
# Preview top 5 predictors for the single/specific prediction
eli5.show_prediction(lig_reg, X_test[1], feature_names=num, show_feature_values=True, top = 10)

In [184]:
# Preview top 5 predictors for the single/specific prediction
eli5.show_prediction(gb_reg, X_test[1], feature_names=num, show_feature_values=True, top = 10)

In [185]:
# We can get the top 15 features as a pandas dataframe
eli5.explain_prediction_df(estimator=lig_reg, doc=X_test[1], top = 15)

,target,feature,weight,value
0,y,<BIAS>,48.647768,1.000000
1,y,x93,8.222554,1.517041
2,y,x173,1.264995,1.683198
3,y,x48,0.664409,0.662634
4,y,x139,0.627324,-0.500341
5,y,x74,0.569858,0.350622
6,y,x42,0.497942,-0.715241
7,y,x220,0.409078,0.710719
8,y,x55,0.369234,-0.044885
9,y,x16,0.353640,-0.336553


In [186]:
# We can get the top 15 features as a pandas dataframe
eli5.explain_prediction_df(estimator=gb_reg, doc=X_test[1], top = 15)

,target,feature,weight,value
0,y,<BIAS>,48.671983,1.000000
1,y,x93,8.810499,1.517041
2,y,x42,1.433806,-0.715241
3,y,x101,0.845671,-0.316668
4,y,x161,0.810327,1.363463
5,y,x215,0.548826,0.524574
6,y,x228,-0.555669,1.271997
7,y,x12,-0.571415,-0.438636
8,y,x88,-0.634980,-0.129556
9,y,x201,-0.718093,1.376752


In [187]:
# Make predictions on the test set
#x = xgb.DMatrix(test[num_cols].fillna(0))
#predsx = xgb_reg.predict(x)
predsl

array([61.24206545, 45.06877433, 47.78110294, ..., 41.73188577,
       41.86305645, 42.93098775])

In [188]:
#predsl = lig_reg.predict(test[num_cols].fillna(0))
test_predgs_mean

array([58.67397569, 46.27864501, 49.8072403 , ..., 38.77860405,
       41.450715  , 46.5398544 ])

In [192]:
mean = np.mean([predsl,test_predgs_mean], axis=0)
mean

array([59.95802057, 45.67370967, 48.79417162, ..., 40.25524491,
       41.65688572, 44.73542107])

In [142]:
# Get the top five features for each predictions
top_predictorsl = []
for i in range(test.shape[0]):
  predictors = eli5.explain_prediction_df(estimator=lig_reg, doc=test[i], top = 15)
  top_predictorsl.append(predictors.feature.tolist())

In [143]:
# Get the top five features for each predictions for xgb
top_predictorsx = []
for i in range(test.shape[0]):
  predictors = eli5.explain_prediction_df(estimator=gb_reg, doc=test[i], top = 15)
  top_predictorsx.append(predictors.feature.tolist())

In [193]:
# Submission file preparation
predictorsl = pd.DataFrame(top_predictorsl, columns = ['feature_' + str(i) for i in range(1,16)])
predictorsl['target'] = mean
predictorsl['child_id'] = ch
predictorsl = predictorsl[ss.columns.tolist()]
predictorsl.head()

,child_id,target,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15
0,ID_0I0999N6S,59.958021,<BIAS>,x93,x220,x60,x12,x118,x214,x144,x121,x161,x23,x182,x141,x13,x185
1,ID_GQ6ONJ4FP,45.673710,<BIAS>,x93,x220,x60,x144,x120,x88,x23,x16,x173,x73,x141,x1,x226,x13
2,ID_YZ76CVRW3,48.794172,<BIAS>,x93,x73,x60,x88,x173,x144,x118,x16,x160,x106,x120,x185,x220,x13
3,ID_BNINCRXH8,65.602458,<BIAS>,x93,x55,x173,x24,x199,x161,x84,x101,x164,x141,x16,x48,x23,x226
4,ID_1U7GDTLRI,41.274730,<BIAS>,x93,x12,x240,x121,x159,x48,x228,x23,x160,x173,x60,x118,x220,x13


In [162]:
# Submission file preparation
predictorsx = pd.DataFrame(top_predictorsx, columns = ['feature_' + str(i) for i in range(1,16)])
predictorsx['target'] = predsl
predictorsx['child_id'] = ch
predictorsx = predictorsx[ss.columns.tolist()]
predictorsx.head()

,child_id,target,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15
0,ID_0I0999N6S,61.123849,<BIAS>,x93,x220,x228,x60,x89,x159,x42,x226,x16,x12,x185,x23,x13,x141
1,ID_GQ6ONJ4FP,45.168574,<BIAS>,x93,x220,x12,x144,x60,x42,x16,x154,x23,x73,x101,x173,x226,x13
2,ID_YZ76CVRW3,47.451443,<BIAS>,x173,x93,x159,x160,x42,x144,x226,x141,x106,x228,x35,x23,x13,x220
3,ID_BNINCRXH8,66.740683,<BIAS>,x93,x13,x42,x173,x96,x15,x1,x73,x101,x118,x139,x141,x48,x161
4,ID_1U7GDTLRI,40.807539,<BIAS>,x93,x12,x42,x73,x144,x60,x164,x160,x23,x118,x159,x226,x220,x13


In [194]:
# Create csv to upload to Zindi for scoring
from google.colab import files

predictorsl.to_csv('BaselineSubmission42.csv', index = False)
files.download('BaselineSubmission42.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##Note
- Please note that you are not restricted to use eli5 for model interpretability. You can use any other open source libraries like:
      - SHAP
      - LIME
      - Yellowbrick
      - Lucid
      - ... and any other